In [ ]:
import copy
import pickle
from pathlib import Path
import warnings
from datetime import datetime
import sys
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from scipy.stats import mannwhitneyu

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

!git clone https://github.com/marcozanchi97/tutorial_meaveas_nn.git

In [ ]:
USER_PATH = 'tutorial_meaveas_nn/'
DATA_PATH = USER_PATH + 'data/'

In [ ]:
df = pd.read_pickle(DATA_PATH + 'data_input_ml_era5_2018_2023.p')

df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Check that the dataset does not have holes
start_date = df['date'].min()
end_date = df['date'].max()
reference_dates = pd.date_range(start=start_date, end=end_date, freq='H')
is_missing_dates = ~reference_dates.isin(df['date'])
assert not is_missing_dates.any(), "There are missing dates in the DataFrame."

print(start_date, end_date)
df.head()

In [ ]:
#spit data into train, validation and test sets

df_train = df[df.date < '2021/01/01']
df_val = df[(df.date >= '2021/01/01') & (df.date < '2022/01/01')]
df_test = df[df.date >= '2022/01/01']

In [ ]:
#normalize data according to mean and std of train set

columns_to_normalize = ['Wind speed', 'Wind direction','Temperature', 'Direct shortwave radiation','Diffuse shortwave radiation',
                        'Total precipitation', 'Humidity'
                       ]

# Initialize dictionaries to store the mean and std for each column
norm_means = {}
norm_stds = {}

# Normalize df_train and save the mean and std
for column in columns_to_normalize:
    norm_means[column] = df_train[column].mean()
    norm_stds[column] = df_train[column].std()
    df_train.loc[:,column] = (df_train[column] - norm_means[column]) / norm_stds[column]

# Normalize df_val and df_test using the mean and std from df_train
for column in columns_to_normalize:
    df_val.loc[:,column] = (df_val[column] - norm_means[column]) / norm_stds[column]
    df_test.loc[:,column] = (df_test[column] - norm_means[column]) / norm_stds[column]

In [ ]:
#create (input,output) examples

def get_data_from_df(df, columns_list, target_name, input_len, prediction_len, shuffle=False, seed_value=None):
    data = df[columns_list].values
    target = df[target_name].values
    X, y = [], []

    for time_idx in range(len(df) - input_len - prediction_len):
        X.append(data[time_idx:time_idx+input_len])
        y.append(target[time_idx+input_len:time_idx+input_len+prediction_len])

    X, y = np.array(X), np.array(y)

    if shuffle:
        rng = np.random.default_rng(seed_value)
        shuffled_indices = rng.permutation(len(X))
        X, y = X[shuffled_indices], y[shuffled_indices]
    return X, y

# Constants
PREDICTION_LENGTH = 24
INPUT_LENGTH = 72
SEED = 123
COLUMNS_LIST = ['Temperature', 'Wind speed', 'Wind direction', 'Direct shortwave radiation',
                'Diffuse shortwave radiation', 'Total precipitation', 'Humidity']
TARGET_NAME = ['Temperature']

# get data examples
X_train, y_train = get_data_from_df(df_train, COLUMNS_LIST, TARGET_NAME, INPUT_LENGTH, PREDICTION_LENGTH,shuffle=True, seed_value=SEED)
X_val, y_val = get_data_from_df(df_val, COLUMNS_LIST, TARGET_NAME, INPUT_LENGTH, PREDICTION_LENGTH,shuffle=True, seed_value=SEED)
X_test, y_test = get_data_from_df(df_test, COLUMNS_LIST, TARGET_NAME, INPUT_LENGTH, PREDICTION_LENGTH)

# Print dataset distributions
print('Train set encompasses {:.1f}% of data'.format(100 * y_train.shape[0] / len(df)))
print('Validation set encompasses {:.1f}% of data'.format(100 * y_val.shape[0] / len(df)))
print('Test set encompasses {:.1f}% of data'.format(100 * y_test.shape[0] / len(df)))


In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# build TSMixer model

def mixer_layer(inputs, ff_dim, activation = 'relu', dropout=0.7):

  norm = layers.BatchNormalization

  # Time mixing
  x = norm(axis=[-2, -1])(inputs)
  x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
  x = layers.Dense(x.shape[-1], activation=activation)(x)
  x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
  x = layers.Dropout(dropout)(x)
  res = x + inputs

  # Feature mixing
  x = norm(axis=[-2, -1])(res)
  x = layers.Dense(ff_dim, activation=activation)(x)  # [Batch, Input Length, FF_Dim]
  x = layers.Dropout(dropout)(x)
  x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
  x = layers.Dropout(dropout)(x)
  return x + res


n_block = 8
ff_dim = 64
activation = 'relu'
dropout = 0.7
pred_len = 24

inputs = tf.keras.Input(shape=X_train[0].shape)


# Residual blocks repetition
x = inputs  # [Batch, Input Length, Channel]
for _ in range(n_block):
  x = mixer_layer(x, ff_dim, activation, dropout)

# Temporal projection
x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

# Output
outputs = layers.Dense(1, activation='linear')(outputs)


model = tf.keras.Model(inputs, outputs)
model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),metrics = ['mae'])
model.summary()

In [ ]:
# train and validate tsmixer

history = model.fit(X_train, y_train,
                    validation_data = (X_val, y_val),
                    epochs= 500,
                    batch_size = 32,
                    callbacks = tf.keras.callbacks.EarlyStopping(patience = 20,
                                                                 monitor = 'val_loss',
                                                                 restore_best_weights = True
                                                                )
                       )


history_df = pd.DataFrame(history.history)

In [ ]:
model.evaluate(X_train,y_train)
model.evaluate(X_val,y_val)
model.evaluate(X_test,y_test)

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds.shape

In [ ]:
#plot some predictions

fig,axs = plt.subplots(5,5, figsize = (10,8))
axs = axs.ravel()

for i in range(len(axs)):
    ax = axs[i]
    idx = random.randint(0, len(preds))

    pred_series = preds[idx,:,:] * norm_stds['Temperature'] + norm_means['Temperature']
    true_series = y_test[idx,:,:] * norm_stds['Temperature'] + norm_means['Temperature']
    ax.plot(pred_series, color = 'red', label = 'prediction')
    ax.plot(true_series, color = 'black', label = 'true')
    ax.set_title('mae: {:.3f}'.format(np.mean(np.abs(pred_series - true_series))))
ax.legend(loc = (1,0))


fig.suptitle('Temperature predictions examples')
fig.tight_layout()

In [ ]:
#plot some results

fig,ax = plt.subplots(1, figsize = (6,4))
ax.set_aspect(1)
x = y_test.reshape(-1)*norm_stds['Temperature'] + norm_means['Temperature']
y = preds.reshape(-1)*norm_stds['Temperature'] + norm_means['Temperature']
sns.regplot(x = x, y = y, ax=ax, truncate=False, color="#007aff")
ax.set_xlabel('Temperature')
ax.set_ylabel('Prediction')
R2 = linregress(x,y).rvalue ** 2
ax.text(0.1, 0.95, f"$R^2 = {R2:.2f}$", fontsize=14, transform=ax.transAxes, va="top", ha="left")
fig.tight_layout()

In [ ]:
input_lenght = 72
prediction_lenght = 24

fig,ax = plt.subplots(1,1, figsize = (12,7))


datalist_dict_preds = []
datalist_dict_base_last_day = []
datalist_dict_avg_3days = []

for idx in range(len(preds)):
  pred_series = preds[idx,:,0] * norm_stds['Temperature'] + norm_means['Temperature']
  true_series = y_test[idx,:,0] * norm_stds['Temperature'] + norm_means['Temperature']
  error = np.abs(pred_series - true_series)

  for time, value in enumerate(error):
    data_dict_preds = {
        'time':time,
        'Temperature': value}
    datalist_dict_preds.append(data_dict_preds)


data = df_test['Temperature'].values* norm_stds['Temperature'] + norm_means['Temperature']

for time_idx in range( len(df_test) - input_lenght - prediction_lenght ):
  baseline_1 = data[time_idx+input_lenght-prediction_lenght:time_idx+input_lenght]
  baseline_2 = data[time_idx+prediction_lenght:time_idx+input_lenght-prediction_lenght]
  baseline_3 = data[time_idx:time_idx+prediction_lenght]

  baseline_last_day = baseline_1
  baseline_3days = np.mean(np.array([baseline_1, baseline_2, baseline_3]), axis = 0)
  target = data[time_idx+input_lenght : time_idx+input_lenght+prediction_lenght]

  base_error_last_day = np.abs(baseline_last_day-target)
  for time, value in enumerate(base_error_last_day):
    data_dict_base_last_day = {
        'time':time,
        'Temperature': value}
    datalist_dict_base_last_day.append(data_dict_base_last_day)


  base_error_3days = np.abs(baseline_3days-target)
  for time, value in enumerate(base_error_3days):
    data_dict_base_3days = {
        'time':time,
        'Temperature': value}
    datalist_dict_avg_3days.append(data_dict_base_3days)



#ax = axs[feat_idx]
df_pred = pd.DataFrame(datalist_dict_preds)
df_base_last_day = pd.DataFrame(datalist_dict_base_last_day)
df_base_avg_3days = pd.DataFrame(datalist_dict_avg_3days)

sns.lineplot(data = df_pred, x = 'time',y= 'Temperature', color = 'darkorange',ax=ax, label = 'TSMixer')
sns.lineplot(data = df_base_last_day, x = 'time',y= 'Temperature', color = 'deepskyblue',ax=ax, label = 'Baseline last day')
sns.lineplot(data = df_base_avg_3days, x = 'time',y= 'Temperature', color = 'limegreen',ax=ax, label = 'Baseline avg 3 days')

fontsize = 16
ax.set_xlabel('Prediction time [Hours]', fontsize = fontsize)
ax.set_ylabel('Absolute error [°C]', fontsize = fontsize)
ax.set_title('Temperature', fontsize = fontsize+2)

ax.tick_params(axis='both', which='both', labelsize=12)
ax.legend(fontsize = 12)



#axs[-1].set_visible(False)

fig.suptitle('Time evolution of absolute error on test set', fontsize = fontsize+5)
fig.tight_layout()